# 03 - Interactive Viz

In [8]:
import pandas as pd
import numpy as np
import json
import folium

1. Go to the eurostat website and try to find a dataset that includes the european unemployment rates at a recent date.



In [40]:
# database by theme/population and social conditions/labour market/
#employment and unemployment/LFS series - detailed annual survey results/Total unemployment/
#Unemployment rates by sex, age and citizen ship(%)

unemp_url =  "http://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=lfsa_ugan&lang=en"
res = get(unemp_url, 'html')

#je sais pas si c'est utile de faire ça 



Use this data to build a Choropleth map which shows the unemployment rate in Europe at a country level. Think about the colors you use, how you decided to split the intervals into data classes or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

In [33]:
# let's build a blank map
m_europe = folium.Map([54.5259614,15.255118700000025], tiles='cartodbpositron', zoom_start=3.6)
m_europe



In [49]:
 # lire les données depuis un autre fichier
unemp_path = 'lfsa_ugan_Data_1.csv'
unemp_data = pd.read_csv(unemp_path) # ça marche pas 

# Chloropleth
m_europe.choropleth(data=unemp_data,
             columns=['Time', 'Geo'], # à compléter??
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
            legend_name='Unemployment rate (%)')


FileNotFoundError: File b'lfsa_ugan_Data_1.csv' does not exist